In [1]:
print("Hey")

Hey


In [1]:
import cv2
import torch
import pyttsx3
import threading
import time
from ultralytics import YOLO
import requests

# Load the YOLOv8 model
model = YOLO('C:/Users/HP/Downloads/best4.pt')

# Initialize webcam capture
cap = cv2.VideoCapture(0)

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 1.0)

# Track the last spoken gesture to avoid repetition
last_spoken_gesture = ""
speech_thread = None

# Performance optimization: resize frames to a smaller size
resize_width = 640
resize_height = 480

# Frame skipping: process every nth frame
frame_skip_interval = 5
frame_count = 0

# Define your NodeMCU IP address
nodemcu_ip = 'http://192.168.137.251'

# Use torch.no_grad() to disable gradient tracking during inference
def detect_hand_gestures(frame):
    with torch.no_grad():
        results = model(frame)
        return results[0]  # Returning the first result assuming single image

def speak_gesture(gesture):
    """Speak the detected gesture in a separate thread."""
    engine.say(gesture)
    engine.runAndWait()

def send_http_request(gesture_name):
    """Send HTTP request to NodeMCU based on the gesture detected."""
    gesture_mapping = {
        'gesture1': 'Hello',       # Turn on Light 1
        'gesture2': 'Thumbs_up',   # Turn off Light 1
        'gesture3': 'Thumbs_down', # Turn on Light 2
        # Add more gestures if needed
    }
    gesture_route = gesture_mapping.get(gesture_name.strip(), '')
    url = f"{nodemcu_ip}/{gesture_route}"
    print(f"Sending request to: {url} '{gesture_name}'")
    try:
        response = requests.get(url)
        print(response.text)  # Optional: Print the server's response
    except Exception as e:
        print(f"Error sending HTTP request: {e}")

while True:
    # Read frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Skip frames for efficiency
    if frame_count % frame_skip_interval != 0:
        continue

    # Resize frame for faster processing
    frame_resized = cv2.resize(frame, (resize_width, resize_height))

    # Perform object detection
    detections = detect_hand_gestures(frame_resized)

    current_gesture = None  # Placeholder for the current detected gesture

    # Draw detections on the frame
    for det in detections:
        boxes = det.boxes.xyxy.numpy()  # Bounding box coordinates
        scores = det.boxes.conf.numpy()  # Confidence scores
        class_ids = det.boxes.cls.numpy()  # Class IDs

        # Draw bounding boxes and labels
        for box, score, cls in zip(boxes, scores, class_ids):
            x1, y1, x2, y2 = map(int, box)
            gesture_name = model.names[int(cls)]  # Gesture label (e.g., 'thumbs up')
            label = f'{gesture_name} {score:.2f}'
            cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_resized, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Set the current detected gesture
            current_gesture = gesture_name

    # If a gesture is detected and it's not the same as the last spoken gesture, speak it
    if current_gesture and current_gesture != last_spoken_gesture:
        # Ensure the previous speech thread has finished before starting a new one
        if speech_thread is None or not speech_thread.is_alive():
            speech_thread = threading.Thread(target=speak_gesture, args=(current_gesture,))
            speech_thread.start()

        # Send HTTP request to NodeMCU
        send_http_request(current_gesture)

        last_spoken_gesture = current_gesture  # Update the last spoken gesture

    # Display the frame with detections
    cv2.imshow('Hand Gesture Recognition', frame_resized)

    # Limit the frame rate by introducing a short sleep (adjust if needed)
    time.sleep(0.05)  # 50 ms delay to reduce CPU/GPU load

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

# Close the text-to-speech engine
engine.stop()


ModuleNotFoundError: No module named 'cv2'